# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
from pyzbar.pyzbar import decode
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import roc_curve, auc

# Load Data

In [ ]:
root_dir = "/mnt/hdd/Datasets/QR codes dataset"
train_dir = "/mnt/hdd/Datasets/QR codes dataset/Train"
valid_dir = "/mnt/hdd/Datasets/QR codes dataset/Val"
test_dir = "/mnt/hdd/Datasets/QR codes dataset/Test"

In [ ]:
test_image = "/mnt/hdd/Datasets/QR codes dataset/Train/benign_10.png"
image = cv2.imread(test_image)
decoded_objects = decode(image)

for obj in decoded_objects:
    data = obj.data.decode("utf-8")
    print(str(data.split()[1]))

In [ ]:
test_image = "/mnt/hdd/Datasets/QR codes dataset/Test/malicious_316801.png"
image = cv2.imread(test_image)
decoded_objects = decode(image)

for obj in decoded_objects:
    data = obj.data.decode("utf-8")
    print(str(data.split()[1]))

In [ ]:
train_data, valid_data, test_data = [], [], []
train_labels, valid_labels, test_labels = [], [], []

for image in os.listdir(train_dir):
    file_path = "Train" + "/" + image
    label = "benign" if "benign" in image else "malicious"
    train_data.append(file_path)
    train_labels.append(label)

for image in os.listdir(valid_dir):
    file_path = "Val" + "/" + image
    label = "benign" if "benign" in image else "malicious"
    valid_data.append(file_path)
    valid_labels.append(label)

for image in os.listdir(test_dir):
    file_path = "Test" + "/" + image
    label = "benign" if "benign" in image else "malicious"
    test_data.append(file_path)
    test_labels.append(label)

In [ ]:
train_df = pd.DataFrame({"filepath": train_data, "label": train_labels})
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.label.value_counts()

In [ ]:
valid_df = pd.DataFrame({"filepath": valid_data, "label": valid_labels})
valid_df.head()

In [ ]:
valid_df.shape

In [ ]:
valid_df.label.value_counts()

In [ ]:
test_df = pd.DataFrame({"filepath": test_data, "label": test_labels})
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df.label.value_counts()

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_gen = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=root_dir,
    x_col="filepath",
    y_col="label",
    batch_size=32,
    shuffle=True,
    class_mode="binary",
    target_size=(128, 128),
    #color_mode="grayscale",
)

valid_gen = datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory=root_dir,
    x_col="filepath",
    y_col="label",
    batch_size=32,
    shuffle=True,
    class_mode="binary",
    target_size=(128, 128),
    #color_mode="grayscale"
)

test_gen = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=root_dir,
    x_col="filepath",
    y_col="label",
    batch_size=32,
    shuffle=True,
    class_mode="binary",
    target_size=(128, 128),
    #color_mode="grayscale"
)

# Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=train_gen.image_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
early = [EarlyStopping(monitor="val_loss", patience=3)]

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=256,
    epochs=10,
    validation_data=valid_gen,
    validation_steps=256,
)

In [ ]:
plt.figure()
plt.title("Model Accuracy")
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="valid")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title("Model Loss")
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="valid")
plt.legend()
plt.show()

In [ ]:
model.save("qr.h5")

In [ ]:
model.evaluate(valid_gen)

# Predict

In [ ]:
valid_gen.reset()
preds = model.predict(valid_gen, verbose=1)

In [ ]:
fpr, tpr, _ = roc_curve(valid_gen.classes, preds)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc :.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.legend(loc="lower right")
plt.show()